In [22]:
import pandas as pd
import plotly.express as px
import re
from fuzzywuzzy import process

/home/jeanlucca/.local/share/virtualenvs/coffee_comp_data-SWVeHkw8/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
coffee_raw_df = pd.read_csv('./coffee_df.csv')

In [13]:
def printDataFrameInfo(df: pd.DataFrame)->None:
    """
    Função que printa as informações do df (df.info())
    e também printa as primeiras 5 linhas do df
    """
    print("--------------------")
    df.info()
    print("--------------------")
    print("--------------------")
    print(df.head())
    print("--------------------")


def convert_to_int_if_possible(col):
    """
    Função para checar se todos os valores da coluna são inteiros, se sim, converte para integer, se não retorna a original
    """
    # checando se todos os valores são inteiros
    if col.apply(float.is_integer).all():
        # Converte para integer
        return col.astype(int)
    else:
        # retorna a coluna original
        return col

## 1- Explorando os dados e primeiras impressões 

In [6]:
printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282 entries, 0 to 2281
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   slug         2282 non-null   object 
 1   all_text     2282 non-null   object 
 2   rating       2282 non-null   int64  
 3   roaster      2282 non-null   object 
 4   name         2282 non-null   object 
 5   location     2281 non-null   object 
 6   origin       2282 non-null   object 
 7   roast        2229 non-null   object 
 8   est_price    2277 non-null   object 
 9   review_date  2282 non-null   object 
 10  agtron       2282 non-null   object 
 11  aroma        2255 non-null   float64
 12  acid         1947 non-null   float64
 13  body         2279 non-null   float64
 14  flavor       2279 non-null   float64
 15  aftertaste   2279 non-null   float64
 16  with_milk    356 non-null    float64
 17  desc_1       2282 non-null   object 
 18  desc_2       2282 non-null 

- ```all_text``` é uma coluna que aparenta ter todas as informações em modo texto, no caso, não é relevante para este propósito
- ```aroma```, ```acid```, ```body```, ```aftertaste```, ```flavor``` - são colunas de avalição 0-5 de propriedades do café
- ```with_milk``` é um avaliação que apresenta muitos dados nulos, e também não achei interessante incluir na comparação, visto que não tomo café com leite :)
- ```agtron```é uma coluna meramente informativa, a respeito da coloração do café torrado - pode ser útil para compor uma página
- ```location``` aparenta ser uma coluna de onde o café foi torrado e embalado. As informações estão bem bagunçadas, misturando cidades, estados, provícias e países
- ```origin``` aparenta ser uma coluna de onde o café foi produzido. As informações estão bem bagunçadas, misturando cidades, estados, provícias e países
- ```est_price``` é um dado interessante para incluir, porém preciso normalizar, visto que possuem moedas diferentes e referências de peso diferentes.
- ```roast``` aparenta ter dados bem normalizados, poderei usar como categorização
- ```desc_x``` aparenta ser uma descrição do café, em modo texto, pode ser útil para compor uma página
- ```rating``` uma classificação de 0 a 100, com números inteiros
- ```slug```contem o link da review de onde os dados foram retirados, útil para redirecionar para o site original
- ```review_date``` é a data em que a review foi publicada, não vejo necessidades de trabalhar com essa informação, que não seja apenas exibi-la
- ```name```diz respeito ao nome do café
- ```roaster``` diz respeito ao nome da empresa que fez a torreifação


## 2- Limpeza e tratamento dos dados
### 2.1 - Plano de ação

- Dropar colunas: ```all_text```, ```with_milk```
- Dropar nulos
- Verificar se existem valores com vírgula, ou apenas inteiros: ```aroma```, ```acid```, ```body```, ```aftertaste```, ```flavor```
- Verificar únicos em ```roast``` e usar como categoria
- Normalizar: ```location```, ```origin```, ```est_price```

In [11]:
# Dropando colunas desejadas
coffee_raw_df.drop(columns=['all_text', 'with_milk'], inplace=True)
printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282 entries, 0 to 2281
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   slug         2282 non-null   object 
 1   rating       2282 non-null   int64  
 2   roaster      2282 non-null   object 
 3   name         2282 non-null   object 
 4   location     2281 non-null   object 
 5   origin       2282 non-null   object 
 6   roast        2229 non-null   object 
 7   est_price    2277 non-null   object 
 8   review_date  2282 non-null   object 
 9   agtron       2282 non-null   object 
 10  aroma        2255 non-null   float64
 11  acid         1947 non-null   float64
 12  body         2279 non-null   float64
 13  flavor       2279 non-null   float64
 14  aftertaste   2279 non-null   float64
 15  desc_1       2282 non-null   object 
 16  desc_2       2282 non-null   object 
 17  desc_3       2280 non-null   object 
dtypes: float64(5), int64(1), ob

In [12]:
# Dropando nulos
coffee_raw_df.dropna(inplace=True)
printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 1908 entries, 2 to 2280
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   slug         1908 non-null   object 
 1   rating       1908 non-null   int64  
 2   roaster      1908 non-null   object 
 3   name         1908 non-null   object 
 4   location     1908 non-null   object 
 5   origin       1908 non-null   object 
 6   roast        1908 non-null   object 
 7   est_price    1908 non-null   object 
 8   review_date  1908 non-null   object 
 9   agtron       1908 non-null   object 
 10  aroma        1908 non-null   float64
 11  acid         1908 non-null   float64
 12  body         1908 non-null   float64
 13  flavor       1908 non-null   float64
 14  aftertaste   1908 non-null   float64
 15  desc_1       1908 non-null   object 
 16  desc_2       1908 non-null   object 
 17  desc_3       1908 non-null   object 
dtypes: float64(5), int64(1), object(

In [14]:
# Convertendo (se possível) alguns tipos
columns_to_check = ['aroma', 'acid', 'body', 'aftertaste', 'flavor']
for col in columns_to_check:
    coffee_raw_df[col] = convert_to_int_if_possible(coffee_raw_df[col])
    
printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 1908 entries, 2 to 2280
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   slug         1908 non-null   object
 1   rating       1908 non-null   int64 
 2   roaster      1908 non-null   object
 3   name         1908 non-null   object
 4   location     1908 non-null   object
 5   origin       1908 non-null   object
 6   roast        1908 non-null   object
 7   est_price    1908 non-null   object
 8   review_date  1908 non-null   object
 9   agtron       1908 non-null   object
 10  aroma        1908 non-null   int64 
 11  acid         1908 non-null   int64 
 12  body         1908 non-null   int64 
 13  flavor       1908 non-null   int64 
 14  aftertaste   1908 non-null   int64 
 15  desc_1       1908 non-null   object
 16  desc_2       1908 non-null   object
 17  desc_3       1908 non-null   object
dtypes: int64(6), object(12)
memory usage: 283.2+ KB
----

In [15]:
# Verificar valores únicos em roast
print(coffee_raw_df['roast'].unique())

['Medium-Light' 'Medium' 'Light' 'Very Dark' 'Medium-Dark' 'Dark']


In [17]:
coffee_raw_df['roast'] = coffee_raw_df['roast'].astype('category')
printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 1908 entries, 2 to 2280
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   slug         1908 non-null   object  
 1   rating       1908 non-null   int64   
 2   roaster      1908 non-null   object  
 3   name         1908 non-null   object  
 4   location     1908 non-null   object  
 5   origin       1908 non-null   object  
 6   roast        1908 non-null   category
 7   est_price    1908 non-null   object  
 8   review_date  1908 non-null   object  
 9   agtron       1908 non-null   object  
 10  aroma        1908 non-null   int64   
 11  acid         1908 non-null   int64   
 12  body         1908 non-null   int64   
 13  flavor       1908 non-null   int64   
 14  aftertaste   1908 non-null   int64   
 15  desc_1       1908 non-null   object  
 16  desc_2       1908 non-null   object  
 17  desc_3       1908 non-null   object  
dtypes: category(

### Limpando ```location``` e ```origin```

In [20]:
## LIMPANDO location e origin

print(coffee_raw_df["location"].value_counts())
print('------------------------')
print(coffee_raw_df["origin"].value_counts())

location
Madison, Wisconsin                        137
Chia-Yi, Taiwan                           108
Minneapolis, Minnesota                    107
San Diego, California                     100
Taipei, Taiwan                             85
                                         ... 
Glendora, California                        1
Songshan District, Taipei City, Taiwan      1
Kountze, Texas                              1
Savannah, Georgia                           1
Bigfork, Montana                            1
Name: count, Length: 272, dtype: int64
------------------------
origin
Yirgacheffe growing region, southern Ethiopia                  77
Guji Zone, Oromia Region, southern Ethiopia                    58
Nyeri growing region, south-central Kenya                      53
Boquete growing region, western Panama                         34
Sidamo (also Sidama) growing region, south-central Ethiopia    32
                                                               ..
Kona growing regio

#### Conclusões
- Percebi que ambas estão no seguinte padrão:
    - Se há o delimitador ";" quer dizer que são duas regiões diferentes, como se os dados estivessem em uma lista. Ex: Brazil;Ecuador;Ethiopia
    - Se há o delimitador "," quer dizer que há uma subregião (menor) e uma região (maior)Ex: Cidade, Estado ou Província, País.
    - Há a combinação de ambos casos. Ex: Bazil; Oromia Region, Ethiopia
- Quero manter apenas os países criando duas colunas novas ```location_country``` e ```origin_country```

In [21]:
def extract_country(location_string:str)->str:
    # Dividindo para pegar as diferentes regiões
    regions = location_string.split(';')
    countries = []
    
    for region in regions:
        # Dividindo por vírgulas e pegando a última parte
        parts = region.split(',')
        countries.append(parts[-1].strip())
    return '; '.join(countries)

coffee_raw_df['location_country'] = coffee_raw_df['location'].apply(extract_country)
coffee_raw_df['origin_country'] = coffee_raw_df['origin'].apply(extract_country)

printDataFrameInfo(coffee_raw_df)

--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 1908 entries, 2 to 2280
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   slug              1908 non-null   object  
 1   rating            1908 non-null   int64   
 2   roaster           1908 non-null   object  
 3   name              1908 non-null   object  
 4   location          1908 non-null   object  
 5   origin            1908 non-null   object  
 6   roast             1908 non-null   category
 7   est_price         1908 non-null   object  
 8   review_date       1908 non-null   object  
 9   agtron            1908 non-null   object  
 10  aroma             1908 non-null   int64   
 11  acid              1908 non-null   int64   
 12  body              1908 non-null   int64   
 13  flavor            1908 non-null   int64   
 14  aftertaste        1908 non-null   int64   
 15  desc_1            1908 non-null   object  
 16  desc_2  

### Conclusões
- Percebi que:
    - Muitos países acompanham termos de direção como southern $Country ou south-central $Country
    - Apesar de termos muitos nomes de países, alguns estão escritos incorretamente
    - Muitos "países" são estados, províncias dos EUA, Canadá, México e etc...
- Logo:
    - Vou fazer uma lista de palavras relacionadas à direção e remover das strings
    - Padronizar os nomes dos países, e usar a biblioteca FuzzyWuzzy para encontrar possíveis erros de digitação e corrigí-los
    - Fazer uma lista de estados e províncias dos EUA, México, Canadá e Taiwan para subsituir pelo nome do país apenas

In [ ]:
def remove_directional_terms(location_string):
    direction_keywords = [
        'southern',
        'northern',
        'eastern',
        'western',
        'south-central',
        'north-central',
        "north-eastern",
        "south-eastern",
        "northeastern",
        "southeastern",
        "far"
        ]
    for word in direction_keywords:
            # Dividindo para pegar as diferentes regiões
        countries = location_string.split(';')
        countries_fixed = []
        for country in countries:
            countries_fixed.append((re.sub(rf'\b{word}\b', '', country, flags=re.IGNORECASE)).strip())
    return countries_fixed.join(';')

coffee_raw_df['location_country'] = coffee_raw_df['location_country'].apply(remove_directional_terms)
coffee_raw_df['origin_country'] = coffee_raw_df['origin_country'].apply(remove_directional_terms)
printDataFrameInfo(coffee_raw_df)